In [1]:
!pip install -r requirements.txt

  Using cached streamlit-1.23.1-py2.py3-none-any.whl.metadata (7.4 kB)
  Using cached langchain-0.0.240-py3-none-any.whl.metadata (14 kB)
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain_huggingface-0.0.3-py3-none-any.whl.metadata (1.2 kB)


ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement langchain-groq==0.0.5 (from versions: 0.0.1, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.8, 0.1.9, 0.1.10, 0.2.0.dev0, 0.2.0.dev1, 0.2.0, 0.2.1, 0.2.2, 0.2.3)
ERROR: No matching distribution found for langchain-groq==0.0.5


In [2]:
import os
import json
from pdfminer.high_level import extract_text
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from langchain.prompts.prompt import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import JSONLoader
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


In [3]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0, 
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [4]:
query = """
    given the information {information} about a person I want you to create:
    1. A short summary
    2. two interesting facts about them
    """

In [5]:
prompt_template = PromptTemplate(
    input_variables=["information"],
    template=query
)

In [6]:
chain = prompt_template | llm

In [7]:

text_data ="""
Geoffrey Everest Hinton (born 6 December 1947) is a British-Canadian computer scientist, cognitive scientist, 
cognitive psychologist, known for his work on artificial neural networks which earned him the title as the 
"Godfather of AI". Hinton is University Professor Emeritus at the University of Toronto. From 2013 to 2023, 
he divided his time working for Google (Google Brain) and the University of Toronto, before publicly announcing 
his departure from Google in May 2023, citing concerns about the risks of artificial intelligence (AI) technology.
In 2017, he co-founded and became the chief scientific advisor of the Vector Institute in Toronto.

With David Rumelhart and Ronald J. Williams, Hinton was co-author of a highly cited paper published in 1986 
that popularised the backpropagation algorithm for training multi-layer neural networks, although they were 
not the first to propose the approach. Hinton is viewed as a leading figure in the deep learning community.
The image-recognition milestone of the AlexNet designed in collaboration with his students Alex Krizhevsky 
and Ilya Sutskever for the ImageNet challenge 2012[22] was a breakthrough in the field of computer vision.

Hinton received the 2018 Turing Award, often referred to as the "Nobel Prize of Computing", together with 
Yoshua Bengio and Yann LeCun, for their work on deep learning. They are sometimes referred to as the 
"Godfathers of Deep Learning", and have continued to give public talks together. He was also awarded 
the 2024 Nobel Prize in Physics, shared with John Hopfield.
"""


In [8]:
output = chain.invoke(input={"information": text_data})

In [9]:
print(output.content)

Here is the information about Geoffrey Everest Hinton:

**Summary:** Geoffrey Everest Hinton is a British-Canadian computer scientist, cognitive scientist, and cognitive psychologist known for his work on artificial neural networks and deep learning. He is often referred to as the "Godfather of AI" and has made significant contributions to the field of computer vision and machine learning.

**Interesting Facts:**

1. **Co-author of a highly cited paper:** Hinton co-authored a paper with David Rumelhart and Ronald J. Williams in 1986 that popularized the backpropagation algorithm for training multi-layer neural networks. This paper has had a significant impact on the field of artificial intelligence.
2. **Recipient of prestigious awards:** Hinton has received several prestigious awards, including the 2018 Turing Award, often referred to as the "Nobel Prize of Computing", and the 2024 Nobel Prize in Physics, shared with John Hopfield. He is also known as the "Godfather of AI" and has bee

In [10]:
pip install jq

Note: you may need to restart the kernel to use updated packages.


In [15]:
def load_json_data(base_path):
    """
    This function loads text data from JSON files organized by topic
    in multiple folders under the specified base path.
    """
    documents = []
    for topic in os.listdir(base_path):
        topic_path = os.path.join(base_path, topic)
        if os.path.isdir(topic_path):
            for filename in os.listdir(topic_path):
                if filename.endswith(".json"):
                    file_path = os.path.join(topic_path, filename)
                    loader = JSONLoader(file_path=file_path, jq_schema=".text")  # Use ".text"
                    documents.extend(loader.load())
    return documents

In [16]:
react_docs = load_json_data(base_path="data/processed")

In [17]:
print(react_docs[0].page_content)

ACCOUNTANT
Summary

Financial Accountant specializing in financial planning, reporting and analysis within the Department of Defense.

Highlights

Account reconciliations
Results-oriented
Financial reporting
Critical thinking

Accomplishments

Accounting operations professional
Analysis of financial systems
ERP (Enterprise Resource Planning) software.
Excellent facilitator

Served on a tiger team which identified and resolved General Ledger postings in DEAMS totaling $360B in accounting adjustments. This allowed
for the first successful fiscal year-end close for 2012.

In collaboration with DFAS Europe, developed an automated tool that identified duplicate obligations. This tool allowed HQ USAFE to
deobligate over $5M in duplicate obligations.

Experience
Company Name July 2011 to November 2012 Accountant 
City , State

Enterprise Resource Planning Office (ERO)

In this position as an Accountant assigned to the Defense Enterprise Accounting and Management System (DEAMS) ERO I was
respo

In [18]:
print(f"number of loaded pages: {len(react_docs)}")

number of loaded pages: 2484


In [23]:
def split_documents(documents, chunk_size=500, chunk_overlap=100):
    """
    Splits documents into chunks, optimized for the provided JSON structure.
    """
    processed_documents = []
    for doc in documents:
        text_content = doc.page_content  # Access the text content directly

        # Add double newlines before headings
        text_content = re.sub(r'(Summary|Skills|Experience)', r'\n\n\1', text_content)

        processed_documents.append(Document(page_content=text_content, metadata=doc.metadata))

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n"] 
    )
    chunks = text_splitter.split_documents(documents=processed_documents)
    return chunks

In [24]:
react_chunks = split_documents(react_docs)

In [25]:
print(f"number of chunks created: {len(react_chunks)}")

number of chunks created: 45566


In [26]:
def create_embedding_vector_db(chunks, db_name):
    """
    this function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a vector database called FAISS, 
    which allows for efficient similarity search
    """
    # instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    # create the vector store 
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding
    )
    # save vector database locally
    vectorstore.save_local(f"../vector_databases/vector_db_{db_name}")

In [27]:
create_embedding_vector_db(chunks=react_chunks, db_name="react")

In [28]:
def retrieve_from_vector_db(vector_db_path):
    """
    this function splits out a retriever object from a local vector database
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
    retriever = react_vectorstore.as_retriever()
    return retriever

In [29]:
react_retriever = retrieve_from_vector_db("../vector_databases/vector_db_react")

In [30]:
type(react_retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [31]:
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

In [32]:
react_retrieval_chain = connect_chains(react_retriever)

In [33]:
output = react_retrieval_chain.invoke(
    {"input": "Give me the summary of ReAct in 5 sentences"}
)

In [34]:
type(output)

dict

In [35]:
output.keys()

dict_keys(['input', 'context', 'answer'])

In [36]:
print(output['answer'])

Based on the provided context, here is a summary of ReAct in 5 sentences:

ReAct commits people to action with timelines. It transforms complex information into easily understood formats.
